# Lab 3

Getting the best models on the planet to generate an SVG

Inspired by the legendary Simon Willison's Pelican riding a bike test

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, SVG
from datetime import datetime
from litellm import acompletion
import asyncio

In [ ]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
grok_api_key = os.getenv('GROK_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if grok_api_key:
    print(f"Grok API Key exists and begins {grok_api_key[:4]}")
else:
    print("Grok API Key not set (and this is optional)")

## Connecting to Python Client libraries

We call Cloud APIs by making REST calls to an HTTP endpoint, passing in our keys.

For convenience, the labs like OpenAI have provided lightweight python client libraries that make the HTTP calls for us.

In [ ]:
# Connect to OpenAI client library
# A thin wrappes around calls to REST endpoints

openai = OpenAI()

# For Gemini, DeepSeek and Groq, we can use the OpenAI python client
# Because Google and DeepSeek have endpoints compatible with OpenAI
# And OpenAI allows you to change the base_url

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"
deepseek_url = "https://api.deepseek.com"
groq_url = "https://api.groq.com/openai/v1"
grok_url = "https://api.x.ai/v1"
ollama_url = 'http://localhost:11434/v1'

anthropic = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)
deepseek = OpenAI(api_key=deepseek_api_key, base_url=deepseek_url)
groq = OpenAI(api_key=groq_api_key, base_url=groq_url)
grok = OpenAI(api_key=grok_api_key, base_url=grok_url)
ollama = OpenAI(base_url=ollama_url, api_key='ollama')


In [ ]:

prompt = "Generate an SVG of a hedgehog wearing airpods. Respond with the SVG only, no code blocks."
messages = [{"role": "user", "content": prompt}]


async def hedgehog(model, effort=None):
    start = datetime.now()
    response = await acompletion(model=model, messages=messages, reasoning_effort=effort)
    result = response.choices[0].message.content
    end = datetime.now()
    elapsed = (end - start).total_seconds()
    heading = f"### {model}\n**Time:** {elapsed // 60:.0f} min {elapsed % 60:.0f} s\n\n"
    return heading, result

In [ ]:
hedgehogs = [
    hedgehog("openai/gpt-5-nano", effort="low"),
    hedgehog("deepseek/deepseek-reasoner"),
    hedgehog("gemini/gemini-2.5-pro", effort="high"),
    hedgehog("xai/grok-4"),
    hedgehog("ollama/gpt-oss:20b", effort="high"),
    hedgehog("groq/openai/gpt-oss-120b"),
    hedgehog("openai/gpt-5", effort="high"),
]

In [ ]:
results = await asyncio.gather(*hedgehogs)

In [ ]:
for result in results:
    try:
        display(Markdown(result[0]))
        display(SVG(result[1]))
    except Exception as e:
        print(f"Error displaying result: {e}")
